## 1. Generate the source dataframes

In [2]:


###### Import packages/modules
import submitit
# memory profiler to evaluate how much your jobs demand
from memory_profiler import memory_usage
# import garbage collector: it is sometimes useful to trigger the garbage collector manually with gc.collect()
import gc
# import other modules
import time

import sys,os
import pickle

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from functools import partial
import importlib   # To import model files with poorly written names  ¯\_(ツ)_/¯
from tabulate import tabulate # to make pretty tables : 

import jax
from jax import vmap
import jax.numpy as jnp
from jax.tree_util import tree_map
import jax.random as jr
import tensorflow_probability.substrates.jax.distributions as tfd

# + local functions : 
from database_handling.database_extract import get_all_subject_data_from_internal_task_id
from utils import remove_by_indices
from analysis_tools.preprocess import get_preprocessed_data_from_df


# The agent class : 
from simulate.general_agent import Agent

# The simulated environment :
from simulate.generate_observations_full_actions import TrainingEnvironment,simulate_training
from simulate.hmm_weights import behavioural_process # The environment is statically defined by its HMM matrices
from simulate.generate_observations_full_actions import generate_synthetic_data

# The methods to predict actions, compute the log-likelihoods and fit the models :
from simulate.compute_likelihood_full_actions import compute_predicted_actions,compute_loglikelihood
from simulate.compute_likelihood_full_actions import fit_mle_agent,fit_map_agent
from simulate.invert_model import invert_data_for_single_model,invert_data_for_library_of_models


def get_results_df(_internal_task_id,_studies_list = None,_exclude_subjects_list = [],
                   _llm_classification_code = None, _llm_classification_file_path = None,
                   _bins_fb_noise = None,
                   _override = False,
                   last_k_trials= 3,last_t_timesteps=5):
    
    if _studies_list is not None :
        # Get a list of the task results, 
        _tasks_results_all = []
        for prolific_study_id in _studies_list:
            task_results = get_all_subject_data_from_internal_task_id(_internal_task_id,prolific_study_id,
                                                                    process_feedback_data_stream=True,override_save=_override)
            print(" - Loaded the task results for study {} \n    ({} subjects.)".format(prolific_study_id,len(task_results)))
            _tasks_results_all += task_results
    else :
        _tasks_results_all = get_all_subject_data_from_internal_task_id(_internal_task_id,
                                                                        process_feedback_data_stream=True,override_save=_override)
        
    print("Total : {} subjects".format(len(_tasks_results_all)))



    # Each subject in task results has the following entries : 
    # TASK_RESULT_FEATURES, TASK_RESULTS_EVENTS, TASK_RESULTS_DATA, TASK_RESULTS,RT_FB
    
    # let's remove some subjects based on broad inclusion criteria : 
    # did not do the task twice, did not revoke the consent midpoint, etc.
    remove_these_subjects = []
    for index,entry in enumerate(_tasks_results_all):
        subj_dict,_,_,_ = entry
        subj_name = subj_dict["subject_id"]
        if subj_name in _exclude_subjects_list:
            remove_these_subjects.append(index)

    _tasks_results_all = remove_by_indices(_tasks_results_all,remove_these_subjects)
    print(str(len(_tasks_results_all)) + " subjects remaining after removing problematic subjects.")
    
    
    
    # Fill a dataframe with that data :
    
    # The initial datframe is the first tuple in our task result list of tuples : 
    subjects_df = pd.DataFrame([entry[0] for entry in _tasks_results_all])

    # Avoid too many categories : 
    subjects_df['Sex'] = np.where(subjects_df['Sex'].isin(['Male','Female']), subjects_df['Sex'], 'Other')

    category_counts = subjects_df['Nationality'].value_counts()
    threshold = 2
    
    
    
    subjects_df['Nationality_red'] = subjects_df['Nationality'].apply(lambda x: x if category_counts[x] >= threshold else 'Other')

    # There was a single noise term for the whole training for each subject : 
    subject_noise_parameters = [np.array(entry[2]["parameters"]["noise_int"])[0] for entry in _tasks_results_all]

    # We add it to the df : 
    subjects_df["feedback_noise_std"] = subject_noise_parameters

    # Time taken to solve the task 
    # Add the time taken recorded by the application : (a better measure than the one provided by Prolific for some reason)
    # subjects_df["application_measured_timetaken"] = (subjects_df["finish_date"]-subjects_df["start_date"]).dt.total_seconds()

    subjects_df["application_measured_timetaken"] = (pd.to_datetime(subjects_df["finish_date"])-pd.to_datetime(subjects_df["start_date"])).dt.total_seconds()
    
    
    # In this dataframe, we're interested in sorting various kinds of data from the trials : 
    # 1/ Data from the instruction phase
    # Load LLM classifications for text responses if they are available !
    if _llm_classification_code is not None :
        classification_instructions = {}
        try : 
            with open(_llm_classification_file_path, 'rb') as f:
                loaded_dict = pickle.load(f)
                
            for question_code,question_contents in loaded_dict.items():
                
                subject_classifs = question_contents["results"][_llm_classification_code]    

                
                subjects_df[question_code] = subject_classifs
                classification_instructions[question_code] = question_contents["prompt"]
                # print(subject_classifs)
            
        except : 
            print("Failed to load LLM classifications.")
    

    # 2/ Data from the feedback gauge :
    # Timestep values :
    all_subject_scores = [subjdata[2]["scoring"] for subjdata in _tasks_results_all]
    subjects_df["raw_feedback_values"] = [subj_scores["feedback"] for subj_scores in all_subject_scores]
    # Real time gauge values :
    subjects_df["realtime_values"] = [subjdata[3][1] for subjdata in _tasks_results_all] # Each element is a list of list os arrays (with varying shape)

    # 3/ Data from the hidden grid :
    # The grid for a specific trial: 
    trial_grids = [entry[2]["process"]["grids"] for entry in _tasks_results_all]
    subjects_df["grid_layout"] = trial_grids
    # Position value :
    subject_positions = [entry[2]["process"]["positions"] for entry in _tasks_results_all]
    subjects_df["subject_positions"] = subject_positions

    goal_positions = [np.array(entry[2]["parameters"]["goal_pos"])[:,0,:] for entry in _tasks_results_all]
    subjects_df["goal_position"] = goal_positions

    def euclidian_distance(position,goal):
        return jnp.linalg.norm(position-goal,2)
    gs = trial_grids[0][0].shape
    maximum_euclidian_dist = euclidian_distance(jnp.array(gs) - jnp.ones((2,)),jnp.zeros((2,)))
    all_euclidian_distances = vmap(vmap(vmap(euclidian_distance,in_axes=(0,None))))(jnp.array(subject_positions),jnp.array(goal_positions))/maximum_euclidian_dist
    subjects_df["norm_distance_to_goal"] = list(all_euclidian_distances)


    # 4/ Data from the realized actions :

    # Actions performed : this encompasses the points dropped
    # But may also include temporal elements such as :
    # - the time taken to perform an actions (first point / second point)
    # - when the action was performed with regard to the gauge
    canvas_size = _tasks_results_all[0][0]["canvas_size"] # Constant across all subjects + conditions
    all_actions_data = np.stack([subjdata[2]["blanket"]["actions"] for subjdata in _tasks_results_all]).astype(float)

    Nsubj,Ntrials,Nactions,Npoints,Nfeatures = all_actions_data.shape
    # print(all_actions_data)
    # Normalize the point data :
    all_actions_data[...,0] = all_actions_data[...,0]/canvas_size[0]
    all_actions_data[...,1] = 1.0 - all_actions_data[...,1]/canvas_size[1]


    # First, let's get a mask for all actions that were NOT performed :
    mask = all_actions_data[...,-1]==1  # values are 1 if the point was recorded
    both_points_only = (mask[...,0] & mask[...,1])
        # All points where at least one value is missing

    Nactions = all_actions_data[...,0,0].size
    Nmissed_actions = (~both_points_only).sum()
    print("A total of {}/{} actions were missed. ({:.2f} %)".format(Nmissed_actions,Nactions,100*Nmissed_actions/Nactions))

    subjects_df["raw_points"] = list(all_actions_data)


    # Encoded barycenters :
    barycenter_x = (all_actions_data[...,0,0]+all_actions_data[...,1,0])/2.0
    barycenter_y = (all_actions_data[...,0,1]+all_actions_data[...,1,1])/2.0
    barycenters = np.stack([barycenter_x,barycenter_y],axis=-1)
    subjects_df["action_barycenters"] = list(barycenters)

    # Encoded euclidian distance between points :
    action_distances = np.linalg.norm(all_actions_data[...,0,:2]-all_actions_data[...,1,:2],axis=-1)
    subjects_df["action_distances"] = list(action_distances)

    # Encoded evolution of point angles :
    angles = np.atan2(all_actions_data[...,1,1]-all_actions_data[...,0,1],all_actions_data[...,1,0]-all_actions_data[...,0,0])
    subjects_df["action_angles"] = list(angles)

    # Encoded delays between stimuli, point1 and point2 :
    all_action_delays = all_actions_data[...,-1,2]
    unfit_actions = (all_action_delays<10)
    subjects_df["action_time_between_points"] = np.where(all_action_delays>10, all_action_delays, np.nan).tolist()

    # Performance metric : we use the average distance to goal state across the lask k_T trials and the last k_t timesteps : (ignoring the blind trial)
    all_distances_to_goal = np.mean(np.stack(subjects_df["norm_distance_to_goal"])[:,-last_k_trials:-1,-last_t_timesteps:],axis=(-1,-2))
    subjects_df["final_performance"] = (1.0 - all_distances_to_goal).tolist()


    # And for the blind trial :
    blind_trial_distances_to_goal = np.mean(np.stack(subjects_df["norm_distance_to_goal"])[:,-1,-last_t_timesteps:],axis=(-1))
    subjects_df["blind_trial_performance"] = (1.0 - blind_trial_distances_to_goal).tolist()
    
    
    
    
    
    # In our situation, the variables of interest are : 
    # a/ The level of noise of the gauge
    # b/ The performance of the subject
    # Let's define broad categories to classify them a bit easier :
    if _bins_fb_noise is None:
        subjects_df['noise_category'] = pd.cut(subjects_df['feedback_noise_std'], bins=[0,0.05,0.15,1.0], labels=["Low", "Medium", "High"])
    else : 
        subjects_df['noise_category'] = pd.cut(subjects_df['feedback_noise_std'], bins=_bins_fb_noise, labels=["Low", "Medium", "High"])
    
    _bins_performance_data = np.linspace(0,1,4)
    subjects_df['performance_category'] = pd.cut(subjects_df['final_performance'], bins=_bins_performance_data, labels=["Poor", "Middling", "Good"])
    
    
    return subjects_df,classification_instructions



# 1. Generate the dataframe we need :
LLM_CLASSIFS_PATH = os.path.join("results","llm_classifications","cluster_classif.data")

STUDIES_EXTRACTION_CODES = {
    "study_1" : {
        "dict_code" : "002",
        "internal_task_id" : "002",
        "studies_id" : None,
        "exclude_subjects" : ["5c9cb670b472d0001295f377"],
        "feedback_noise_bins" : [0,0.2,0.4,1]
    },
    "study_2" : {
        "dict_code" : "003_1",
        "internal_task_id" : "003",
        "studies_id" :  ["66f96c31e69227986334a027","66d086503c0a69291c361b67"],
        "exclude_subjects" : ["615c1741d4630b25e6bc1cb9"],
        "feedback_noise_bins" : [0,0.05,0.15,1]
    },
    "study_3" : {
        "dict_code" : "003_2",
        "internal_task_id" : "003",
        "studies_id" : ["6703ab18d345eaa4893587e0","66f9aee8210357265a5958fc","6703ab1a7ea30557549dc6da"],
        "exclude_subjects" : ["611d60c383f4f70ff4bc99fd", "66a74bdfdcaccdc0703894d5",
                              "667d92f2ea5c1542f417285d", "6548f570022275786186ffbd"],
        "feedback_noise_bins" : [0,0.05,0.15,1]
    }
}




full_dataframe = pd.DataFrame()
for study_name,study_codes in STUDIES_EXTRACTION_CODES.items() :

    dataframe,qsts = get_results_df(study_codes["internal_task_id"],_studies_list = study_codes["studies_id"],_exclude_subjects_list = study_codes["exclude_subjects"],
                    _llm_classification_code = study_codes["dict_code"], _llm_classification_file_path = LLM_CLASSIFS_PATH,
                    _bins_fb_noise = study_codes["feedback_noise_bins"],
                    _override = False)
    dataframe["study_name"] = study_name
    
    full_dataframe = pd.concat([full_dataframe,dataframe],ignore_index=True)
    
print(full_dataframe.columns)


Total : 90 subjects
89 subjects remaining after removing problematic subjects.
Failed to load LLM classifications.
A total of 1557/9790 actions were missed. (15.90 %)
 - Loaded the task results for study 66f96c31e69227986334a027 
    (16 subjects.)
 - Loaded the task results for study 66d086503c0a69291c361b67 
    (40 subjects.)
Total : 56 subjects
55 subjects remaining after removing problematic subjects.
Failed to load LLM classifications.
A total of 14/6050 actions were missed. (0.23 %)
 - Loaded the task results for study 6703ab18d345eaa4893587e0 
    (49 subjects.)
 - Loaded the task results for study 66f9aee8210357265a5958fc 
    (50 subjects.)
 - Loaded the task results for study 6703ab1a7ea30557549dc6da 
    (50 subjects.)
Total : 149 subjects
145 subjects remaining after removing problematic subjects.
Failed to load LLM classifications.
A total of 33/15950 actions were missed. (0.21 %)
Index(['subject_id', 'task_code', 'lang', 'browser', 'N_trials', 'N_tmstps',
       'start_d

## 2. Preprocess the data 

In [6]:
inverted_dataframe = full_dataframe[(full_dataframe["study_name"]=="study_2")|(full_dataframe["study_name"]=="study_3")]

# environment constants :
NTRIALS = 10
T = 11

# MODEL CONSTANTS :
N_LATENT_STATES = 5

# ENVIRONMENTAL CONSTANTS :
N_FEEDBACK_OUTCOMES = 10
TRUE_FEEDBACK_STD =  0.175#0.025
GRID_SIZE = (7,7)
START_COORD = [[5,1],[5,2],[4,1]]
END_COORD = [0,6]

# The weights of the HMM environment
(a,b,c,d,e,u),fb_vals = behavioural_process(GRID_SIZE,START_COORD,END_COORD,N_FEEDBACK_OUTCOMES,TRUE_FEEDBACK_STD)

rngkey = jax.random.PRNGKey(np.random.randint(0,10))
ENVIRONMENT = TrainingEnvironment(a,b,c,d,e,u,T,NTRIALS)

# define the static dimensions of the problem :
No = N_FEEDBACK_OUTCOMES
Ns = N_LATENT_STATES
MODEL_CONSTANTS = {
    "position" : {
        "Th" : 2,
        "N_actions" : 9,
        "N_outcomes" : No,
        "N_states" : Ns
    },
    "angle" : {
        "Th" : 2,
        "N_actions" : 9,
        "N_outcomes" : No,
        "N_states" : Ns
    },
    "distance" : {
        "Th" : 2,
        "N_actions" : 4,
        "N_outcomes" : No,
        "N_states" : Ns
    },
}


# If the points were too close, no angle was recorded :
# The limit was arbitrarily chosen at 7.5 units :
min_dist_norm = 7.5/(np.sqrt(2)*750)

preprocessing_options = {
    "actions":{
        "distance_bins" : np.array([0.0,min_dist_norm,0.2,0.5,jnp.sqrt(2) + 1e-10]),
        "angle_N_bins"  : 8,
        "position_N_bins_per_dim" : 3
    },
    "observations":{
        "N_bins" : N_FEEDBACK_OUTCOMES,
        "observation_ends_at_point" : 2
    }
}
# We can modify these at will


# The preprocessing here assumes two possible observations : 
# - the mean of the gauge level between subject actions (index 0) 
# - the last value before subject action (index 1).
inverted_data_all = get_preprocessed_data_from_df(inverted_dataframe,
                            preprocessing_options,
                            verbose=True,
                            autosave=True,autoload=True,override_save=True,
                            label="default",
                            filter_angles_if_small_distance = True)


formatted_stimuli= [inverted_data_all["observations"]["vect"][1]]
bool_stimuli = [jnp.ones_like(stim[...,0]) for stim in formatted_stimuli] # Everything was seen :)
rewards = jnp.array(inverted_data_all["observations"]["deltas"])

actions = {}
for action_dim in ["position","distance","angle"]:
    actions[action_dim] = jnp.array(inverted_data_all["actions"]["vect"][action_dim])[:,:-1,:,:] # Ignore the last trial

Nsubj,Ntrials,Nactions,_ = actions[action_dim].shape
timesteps = jnp.broadcast_to(jnp.arange(Nactions+1),(Nsubj,Ntrials,Nactions+1))

inverted_data = (formatted_stimuli,bool_stimuli,rewards,actions,timesteps)



formatted_stimuli,_,_,_,_ = inverted_data
Nsubj,Ntrials,Ntimesteps,_ = formatted_stimuli[0].shape

Out of the 22000.0 actions performed by our subjects, 21953.0 were 'valid' (99.8 %)
Out of the 22000.0 feedback sequences potentially observed by our subjects, 22000 were 'valid' (100.0 %)


## 3. Model space

Here, we define all the models we want to check against our subject datas !

In [8]:

invert_these = []

# RW models with no modality selection : 
for free_parameters in ["independent","mixed"]:
    for assymetric_learning_rate in [True,False] :
        for biases in [[],["static"],["initial"],["static","initial"]]:
            invert_these.append({
                "model_family" : "rw",  # "rw" / "latql" / "tracking_rw"
                "free_parameters" : free_parameters,
                "biaises" : biases,
                "assymetric_learning_rate" : assymetric_learning_rate,
                "modality_selector" : None,
                "generalizer" : False,
                "memory" : "5000M",
            })

# RW models with modality selection :
for free_parameters in ["independent","mixed"]:
    for assymetric_learning_rate in [True,False] :
        for biases in [[],["static"],["initial"],["static","initial"]]:
            for focused_learning in [True,False]:
                for independent_focused_learning in [True,False]:
                    invert_these.append({
                        "model_family" : "rw",  # "rw" / "latql" / "tracking_rw"
                        "free_parameters" : free_parameters,
                        "biaises" : biases,
                        "assymetric_learning_rate" : assymetric_learning_rate,
                        "modality_selector" : {
                            "learn" : True,
                            "metric" : "q_value", #""js_controll", "q_value","surprisal"
                            "biaises" : ["initial"],
                            "focused_learning" : focused_learning,
                            "independent_focused_learning_weights" : independent_focused_learning
                        },
                        "generalizer" : False,
                        "memory" : "5000M",
                    })


# TRW models with modality selection :
for free_parameters in ["independent","mixed"]:
    for assymetric_learning_rate in [True,False] :
        for biases in [[],["static"],["initial"],["static","initial"]]:
            for focused_learning in [True,False]:
                for independent_focused_learning in [True,False]:
                    invert_these.append({
                        "model_family" : "trw",  # "rw" / "latql" / "tracking_rw"
                        "free_parameters" : free_parameters,
                        "biaises" : biases,
                        "assymetric_learning_rate" : assymetric_learning_rate,
                        "modality_selector" : {
                            "learn" : True,
                            "metric" : "js_controll", #""js_controll", "q_value","surprisal"
                            "biaises" : ["initial"],
                            "focused_learning" : focused_learning,
                            "independent_focused_learning_weights" : independent_focused_learning
                        },
                        "generalizer" : False,
                        "memory" : "40000M",
                    })
                    
# latql models with modality selection :
for free_parameters in ["independent","mixed"]:
    for assymetric_learning_rate in [True,False] :
        for biases in [[],["static"],["initial"],["static","initial"]]:
            for focused_learning in [True,False]:
                for independent_focused_learning in [True,False]:
                    invert_these.append({
                        "model_family" : "latql",  # "rw" / "latql" / "tracking_rw"
                        "free_parameters" : free_parameters,
                        "biaises" : biases,
                        "assymetric_learning_rate" : assymetric_learning_rate,
                        "modality_selector" : {
                            "learn" : True,
                            "metric" : "js_controll", #""js_controll", "q_value","surprisal"
                            "biaises" : ["initial"],
                            "focused_learning" : focused_learning,
                            "independent_focused_learning_weights" : independent_focused_learning
                        },
                        "generalizer" : False,
                        "memory" : "40000M",
                    })


# aif models with modality selection based on js_controllability :
for free_parameters in ["independent","mixed"]:
    for biases in [[],["initial"]]:
        for learn_during_trials in [True,False]:
            for learn_habits in [True,False]:
                invert_these.append({
                    "model_family" : "aif",  # "rw" / "latql" / "tracking_rw"
                    "free_parameters" : free_parameters,
                    "biaises" : biases,
                    "learn_habits" : learn_habits,
                    "learn_during_trials" :learn_during_trials,
                    "modality_selector" : {
                        "learn" : True,
                        "metric" : "js_controll", #""js_controll", "q_value","surprisal","efe"
                        "biaises" : ["initial"],
                        "focused_learning" : False,
                        "independent_focused_learning_weights" : False
                    },
                    "generalizer" : False,
                    "memory" : "40000M",
                })



memories = [m["memory"] for m in invert_these]
agents = [Agent(m,MODEL_CONSTANTS) for m in invert_these]
MODEL_LIBRARY = {a.get_name() : {"agent" :a,"tags" : a.get_tags(),"memory":m} for (a,m) in zip(agents,memories)}
print(MODEL_LIBRARY.keys())


print("Inverting a total of {} models.".format(len(MODEL_LIBRARY)))

dict_keys(['i_rwa', 'i_rwa+b', 'i_rwa&b', 'i_rwa+b&b', 'i_rw', 'i_rw+b', 'i_rw&b', 'i_rw+b&b', 'm_rwa', 'm_rwa+b', 'm_rwa&b', 'm_rwa+b&b', 'm_rw', 'm_rw+b', 'm_rw&b', 'm_rw+b&b', 'i_rwa_omegaq&b+fla', 'i_rwa_omegaq&b+fl', 'i_rwa_omegaq&b', 'i_rwa+b_omegaq&b+fla', 'i_rwa+b_omegaq&b+fl', 'i_rwa+b_omegaq&b', 'i_rwa&b_omegaq&b+fla', 'i_rwa&b_omegaq&b+fl', 'i_rwa&b_omegaq&b', 'i_rwa+b&b_omegaq&b+fla', 'i_rwa+b&b_omegaq&b+fl', 'i_rwa+b&b_omegaq&b', 'i_rw_omegaq&b+fla', 'i_rw_omegaq&b+fl', 'i_rw_omegaq&b', 'i_rw+b_omegaq&b+fla', 'i_rw+b_omegaq&b+fl', 'i_rw+b_omegaq&b', 'i_rw&b_omegaq&b+fla', 'i_rw&b_omegaq&b+fl', 'i_rw&b_omegaq&b', 'i_rw+b&b_omegaq&b+fla', 'i_rw+b&b_omegaq&b+fl', 'i_rw+b&b_omegaq&b', 'm_rwa_omegaq&b+fla', 'm_rwa_omegaq&b+fl', 'm_rwa_omegaq&b', 'm_rwa+b_omegaq&b+fla', 'm_rwa+b_omegaq&b+fl', 'm_rwa+b_omegaq&b', 'm_rwa&b_omegaq&b+fla', 'm_rwa&b_omegaq&b+fl', 'm_rwa&b_omegaq&b', 'm_rwa+b&b_omegaq&b+fla', 'm_rwa+b&b_omegaq&b+fl', 'm_rwa+b&b_omegaq&b', 'm_rw_omegaq&b+fla', 'm_rw_om

In [ ]:
# Check for duplicates :
namelist = list(MODEL_LIBRARY.keys())
print(len(namelist) == len(set(namelist)))


# Get a typical hyperparameter dictionnary for each model :
parameter_count_list = []
for model_name,model_contents in MODEL_LIBRARY.items():   
    parameter_set_tree = model_contents["agent"].get_random_parameters(jr.PRNGKey(0))

    vls,_ = (jax.tree.flatten(tree_map(lambda x : x.size,parameter_set_tree)))    
    
    nbr,_ = (jax.tree.flatten(tree_map(lambda x : x.shape[0],parameter_set_tree)))    
    
    parameter_count_list.append([model_name,sum(vls),sum(nbr)])#"{}({:.1f} %)".format(sum(nbr),(sum(nbr)+1e-10)/(sum(vls)+1e-10)*100)])

print(tabulate(parameter_count_list, headers=['model name', '# of parameters \n(scalar values)','# of parameters \n(distinct variables)'], tablefmt='orgtbl'))   
 

True
| model name            |   # of parameters  |       # of parameters  |
|                       |    (scalar values) |   (distinct variables) |
|-----------------------+--------------------+------------------------|
| random                |                  0 |                      0 |
| i_rw                  |                  6 |                      6 |
| i_rwa                 |                  9 |                      9 |
| i_rw+b                |                 31 |                     12 |
| i_rwa+b               |                 34 |                     15 |
| i_rw&b                |                 28 |                      9 |
| i_rwa&b               |                 31 |                     12 |
| i_rw&b_directq+fl     |                 29 |                     10 |
| i_rw&b_directq        |                 29 |                     10 |
| i_rw+b_directq+fl     |                 32 |                     13 |
| i_rw+b_directq        |                 32 |             

We want to parrallelize the inversion of each of those models on the cluster ! 
The bigger ones have a significant computational overhead making them unfit for parrallelization. Thus, we perform the inversion subject per subject. The smaller ones (rw models) can be inverted in parallel :

In [ ]:
# for model_name,model_contents in MODEL_LIBRARY.items():
#     model_contents["n_heads"] = 20
#     model_contents["n_steps"] = 750
    
#     # RW models are quite cheap to invert :
#     if "rw" in model_contents["tags"]:
#         model_contents["vectorize_fit"] = True 
    
#     # In the absence of parrallelized computation, the latent Q-Learning models are quite expensive to invert :
#     if "latql" in model_contents["tags"]:
#         model_contents["vectorize_fit"] = True 

In [ ]:
EXECUTOR_LOGPATH = os.path.join("results","cluster_inversions","logs")
RESULTS_SAVEPATH = os.path.join("results","cluster_inversions","inversion_mle")
if not os.path.exists(RESULTS_SAVEPATH):
    os.makedirs(RESULTS_SAVEPATH)
print(os.path.join(os.getcwd(),EXECUTOR_LOGPATH))


n_jours_max = 7
JOB_MAXTIME = 60*24*n_jours_max
JOB_CPUS = 8
JOB_PARTITION = 'CPU'
JOB_NODES = 1
JOB_NAME_FUNC = (lambda x : "BRUH_{}".format(x))

method = 'mle'

init_lr = 1e-1
n_steps = 1500
n_heads = 20
lr_scheduler = {250:init_lr/2,1000:init_lr/10}
# RESULTS = invert_data_for_library_of_models(data_all_subjects,reduced_library,method="mle",
#                                   standard_n_heads=n_heads,standard_n_steps=n_steps,lr=init_lr,lr_scheduler=lr_scheduler,
#                                   rngkey=jr.PRNGKey(100),
#                                   save=True,save_directory=RESULTS_SAVEPATH,override=False)

rngkey = jr.PRNGKey(0)
option_verbose = True

model_library =MODEL_LIBRARY# {'random' : MODEL_LIBRARY["random"]}
print(model_library)

# loop over array_parallel
print('#### Start submitting jobs #####')

# initialize a list in which our returning jobs will be stored
joblist=[]

for i,(agent_name, agent_contents) in enumerate(model_library.items()):
    print("     -> Agent : {}".format(agent_name))

    # executor is the submission interface (logs are dumped in the folder)
    # note that the executor was modified since the first version of this tutorial (it used to be AutoExecutor).
    # This has implications for the key-value pairs that can be passed to update_parameters.
    executor = submitit.SlurmExecutor(folder=os.path.join(os.getcwd(),EXECUTOR_LOGPATH), max_num_timeout=5)
    
    # define execution parameters (see below for a list of common options)
    executor.update_parameters(mem=agent_contents["memory"],cpus_per_task=JOB_CPUS, time=JOB_MAXTIME, partition =JOB_PARTITION,nodes=JOB_NODES, job_name=JOB_NAME_FUNC(agent_name))

    rngkey,local_key = jr.split(rngkey)
    local_savepath = os.path.join(RESULTS_SAVEPATH,agent_name)
    job_function = partial(invert_data_for_single_model,method=method,
                        standard_n_heads = n_heads,standard_n_steps = n_steps,lr=init_lr,lr_scheduler=lr_scheduler,
                        rngkey = local_key,option_verbose = option_verbose,
                        save=True,save_directory=local_savepath,override=False)   

    # actually submit the job: note that "agent_contents" correspond to that of the inverted model in this iteration
    job = executor.submit(job_function, inverted_data, agent_contents)
    
    # add info about job submission order
    job.job_initial_indice=i 
          
    # print the ID of your job
    print("     >>> submit job" + str(job.job_id))  
    
    # append the job to the joblist
    joblist.append(job)

### wait for jobs to return (in the correct submission order)
print('#### Start waiting for jobs to return #####')
finished_results = [job.result() for job in joblist]

### display jobs
print('#### All jobs completed #####')
print(len(finished_results))
    

/home/come.annicchiarico/code/behavioural_task/Behavioural_analog_to_BCI_training/results/cluster_inversions/logs
{'random': {'model': functools.partial(<function agent at 0x7fc801425580>, constants={'position': {'N_actions': 9, 'N_outcomes': 6, 'N_states': 5}, 'angle': {'N_actions': 9, 'N_outcomes': 6, 'N_states': 5}, 'distance': {'N_actions': 4, 'N_outcomes': 6, 'N_states': 5}}), 'ranges': {'angle': None, 'position': None, 'distance': None}, 'priors': {'angle': None, 'position': None, 'distance': None}, 'bypass': True, 'tags': ['random'], 'n_heads': 20, 'n_steps': 750}, 'i_rw': {'model': functools.partial(<function agent at 0x7fc801427380>, constants={'position': {'N_actions': 9, 'N_outcomes': 6, 'N_states': 5}, 'angle': {'N_actions': 9, 'N_outcomes': 6, 'N_states': 5}, 'distance': {'N_actions': 4, 'N_outcomes': 6, 'N_states': 5}}), 'ranges': {'angle': {'alpha_Q': Array([-10,  10], dtype=int32), 'beta_Q': Array([-3,  3], dtype=int32)}, 'position': {'alpha_Q': Array([-10,  10], dtype=